In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Markdown, Latex
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
import warnings

## Library for more complex plots
import plotly.figure_factory as ff

## For subplots
from plotly import subplots 

import cufflinks as cf
%matplotlib inline

display(HTML("""
<style>
.output {
    display: flex;
    align-items: center;
    text-align: center;
    
}
div.output_subarea{
    max-width:1200px;
}
div.text_cell_render{
padding: 5em 5em 0.5em 0.5em
}

</style>
"""))

## [QSAR Fish Toxicity](https://archive.ics.uci.edu/ml/datasets/QSAR+fish+toxicity#)
The following data was used to generate a model to predict aquatic toxicity. 

The response variable is LC50, which causes death in 50% of test fish over a duration of 96 hours.

The other variables (molecular descriptors) are:

1. CICO (information indices)
2. SM1_Dz(Z) (2D matrix-based descriptors)
3. GATS1i (2D autocorrelations)
4. NdsCH (atom-type counts)
5. NdssC (atom-type counts)
6. MLOGP (molecular properties)


In [6]:
df = pd.read_csv('qsar_fish_toxicity.csv', sep=';' ,
                 names = 'CIC0 SM1_Dz(Z) GATS1i NdsCH NDssC MLOGP LC50'.split(' '))

### Below is a high level view of the whole dataset.
- The dataset contains seven columns and 908 rows. 

In [8]:
df.head()

,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NDssC,MLOGP,LC50
0,3.260,0.829,1.676,0,1,1.453,3.770
1,2.189,0.580,0.863,0,0,1.348,3.115
2,2.125,0.638,0.831,0,0,1.348,3.531
3,3.027,0.331,1.472,1,0,1.807,3.510
4,2.094,0.827,0.860,0,0,1.886,5.390


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 7 columns):
CIC0         908 non-null float64
SM1_Dz(Z)    908 non-null float64
GATS1i       908 non-null float64
NdsCH        908 non-null int64
NDssC        908 non-null int64
MLOGP        908 non-null float64
LC50         908 non-null float64
dtypes: float64(5), int64(2)
memory usage: 49.7 KB


In [10]:
df.describe()

,CIC0,SM1_Dz(Z),GATS1i,NdsCH,NDssC,MLOGP,LC50
count,908.000000,908.000000,908.000000,908.000000,908.000000,908.000000,908.000000
mean,2.898129,0.628468,1.293591,0.229075,0.485683,2.109285,4.064431
std,0.756088,0.428459,0.394303,0.605335,0.861279,1.433181,1.455698
min,0.667000,0.000000,0.396000,0.000000,0.000000,-2.884000,0.053000
25%,2.347000,0.223000,0.950750,0.000000,0.000000,1.209000,3.151750
50%,2.934000,0.570000,1.240500,0.000000,0.000000,2.127000,3.987500
75%,3.407000,0.892750,1.562250,0.000000,1.000000,3.105000,4.907500
max,5.926000,2.171000,2.920000,4.000000,6.000000,6.515000,9.612000


### The scatter matrix below shows a clear positive relationship between LC50 and MLOGP. There may also be a positive relationship between LC50 and CICO

In [77]:

# df.iloc[:,:3].scatter_matrix(dimensions = list(df.columns)[:3])

px.scatter_matrix(df, opacity=0.5, height = 1000)
# fig = go.Figure(data=go.Splom(
#         dimensions={}
#     ))

### The heatmap below shows the relatioship between MGOP and LC50 a bit more clearly. 

In [66]:
data = [go.Heatmap(x=df.columns, y=df.columns,z=df.corr(), colorscale="Viridis")]

layout = go.Layout(title='Correlations')
fig = go.Figure(data, layout)
fig.show()

### Most of the histograms of the features appear to be normally distributed. 
Select a histogram to focus by clicking on the legend items.
* Some items to note
    - NdsCH and NdsCH (atom type counts) are categorical variables that have a skew with a lot of zero values. 

In [73]:
df.iplot(kind='hist', bins=50, title='Histograms of all the features')

### The scatter plot below shows a clear relationship between a higher value of NdsCH (denoted by the size of the point) and LG50

In [83]:
data = go.Scatter(y=df['LC50'], x=df['MLOGP'], mode='markers',
                 marker=dict(size=df['NdsCH']*5+10, color=df['NDssC'],
                            showscale=True))

layout = go.Layout(title='LC50 and MLOGP based on NdsCH(size) and NDssc(color)',
                  xaxis=dict(title='MLOGP'),
                  yaxis=dict(title='LG50'))

fig = go.Figure(data, layout)

fig.show()

### Visual analysis from above provides evidence that there may be a positive relationship between MLOGP plus NdsCH and LG50.
- Data will first be split 70-30 into a training and test set and then I will create a Linear Regression model using the training set.

In [85]:
X = df.drop('LC50', axis=1)
y = df['LC50']

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from scipy import stats

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

### The performance of the model shows several key points
- The Adjusted $R^2$ of the model shows that I am able to explain about 57% of the variability in LC50
- All the variables have p-value lower than $\alpha=0.05$, except NDssC
    - Therefore, it is possible to achieve a more parsimonious model by dropping NDssC


In [109]:
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
warnings.filterwarnings(action='once')
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                   LC50   R-squared:                       0.577
Model:                            OLS   Adj. R-squared:                  0.574
Method:                 Least Squares   F-statistic:                     205.0
Date:                Tue, 31 Dec 2019   Prob (F-statistic):          1.33e-164
Time:                        13:28:41   Log-Likelihood:                -1238.0
No. Observations:                 908   AIC:                             2490.
Df Residuals:                     901   BIC:                             2524.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.1746      0.181     12.000      0.0

### The summary for the final model (without NDssC) is below.
- The final model does not see a decrease in performance (adjusted $R^2$ does not fall)

In [111]:
X2_drop = sm.add_constant(X.drop('NDssC', axis=1))
est = sm.OLS(y, X2_drop)
est3 = est.fit()
warnings.filterwarnings('ignore')
print(est3.summary())

                            OLS Regression Results                            
Dep. Variable:                   LC50   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.574
Method:                 Least Squares   F-statistic:                     245.1
Date:                Tue, 31 Dec 2019   Prob (F-statistic):          2.81e-165
Time:                        13:28:48   Log-Likelihood:                -1239.3
No. Observations:                 908   AIC:                             2491.
Df Residuals:                     902   BIC:                             2519.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.1112      0.177     11.935      0.0

### The Residuals vs. Fitted plot appears to be normal, which suggest that the model is defined correctly. 

In [122]:
residuals = est3.resid
fitted = est3.fittedvalues


data = go.Scatter(y=residuals, x=fitted, mode='markers',
                 marker=dict(size=12, color='orange', opacity=0.5))

layout = go.Layout(title='Residuals vs. Fitted',
                  xaxis=dict(title='Fitted'),
                  yaxis=dict(title='Residuals'))

fig = go.Figure(data, layout)

fig.show()

In [156]:
pd.DataFrame(est3.params, columns=['Parameters'])


,Parameters
const,2.111215
CIC0,0.417451
SM1_Dz(Z),1.299750
GATS1i,-0.752218
NdsCH,0.431526
MLOGP,0.379630
